In [ ]:
!pip install auto-gptq
!pip install optimum
!pip install -i https://pypi.org/simple/ bitsandbytes
!pip install accelerate

In [ ]:
# resolving "No inf checks were recorded for this optimizer." issue
!pip uninstall torch -y
!pip install torch==2.1

# Load libraries

In [1]:
from huggingface_hub import notebook_login
notebook_login()

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from peft import prepare_model_for_kbit_training
from peft import LoraConfig, get_peft_model
from datasets import load_dataset
import transformers
import torch

# Load model

In [3]:
# Model name
model_name = "meta-llama/Llama-2-7b-hf"

# Config to load model in 4bits
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Initiate model to compare effect of fine tuning
pretrained_model = AutoModelForCausalLM.from_pretrained(model_name,
                                             device_map="auto",
                                             quantization_config=bnb_config,
                                             trust_remote_code=True,
                                             revision="main",
                                             is_decoder=True)

# Model for fine tuning
model = AutoModelForCausalLM.from_pretrained(model_name,
                                             device_map="auto",
                                             quantization_config=bnb_config,
                                             trust_remote_code=True,
                                             revision="main",
                                             is_decoder=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

# Load tokenizer from model

In [4]:
# Load tokenizer used in the model
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

# Test base model

In [5]:
# Model architecture
print(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRM

In [6]:
pretrained_model.eval()

# craft prompt
problem = "I can't sleep at night, what should i do?"

# Llama 2 recommended prompt
prompt = f"""<s>[INST] <<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
<</SYS>>

{problem} [/INST]"""

# Tokenizer
inputs = tokenizer(prompt, return_tensors="pt")

# Text generation
outputs = pretrained_model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=140)

print(tokenizer.batch_decode(outputs)[0])

<s><s> [INST] <<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
<</SYS>>

I can't sleep at night, what should i do? [/INST]

[INST] <<SYS>>
I can't sleep at night, what should i do?
<</SYS>>

[INST] <<SYS>>
I can't sleep at night, what should i do?
<</SYS>>

[INST] <<SYS>>
I can't sleep at night, what should i do?
<</SYS>>

[INST] <<SYS>>
I can't sleep at night, what should i do?
<</SYS>>

[INST] <<SYS>>
I can't sleep at night, what should i do?



# Prepare model for training

In [7]:
model.train()

model.gradient_checkpointing_enable()

model = prepare_model_for_kbit_training(model)

In [8]:
# LoRA config
config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["o_proj", "q_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# LoRA trainable version of model
model = get_peft_model(model, config)

model.print_trainable_parameters()

trainable params: 19,988,480 || all params: 6,758,404,096 || trainable%: 0.2957573965106688


# Preparing traning dataset

In [9]:
# load dataset
data = load_dataset("Amod/mental_health_counseling_conversations", split="train")

# Train evaluation data split
data = data.train_test_split(test_size=0.3)

Generating train split: 0 examples [00:00, ? examples/s]

In [10]:
data

DatasetDict({
    train: Dataset({
        features: ['Context', 'Response'],
        num_rows: 2458
    })
    test: Dataset({
        features: ['Context', 'Response'],
        num_rows: 1054
    })
})

In [11]:
# create tokenize function
def tokenize_function(features):

  context = features["Context"]
  response = features["Response"]

  combined_text = f""" problem: {context} ¥n
  response: {response}
  """

  tokenizer.truncation_side = "left"

  tokenizer.pad_token = tokenizer.eos_token
  tokenized_inputs = tokenizer(
      combined_text,
      return_tensors="np",
      padding=True,
  )

  tokenized_inputs = tokenizer(
      combined_text,
      return_tensors="np",
      truncation=True,
      max_length=512

  )

  return tokenized_inputs

tokenized_data = data.map(tokenize_function, batched=True, batch_size=10, drop_last_batch=True, remove_columns=["Context", "Response"])

tokenized_data

Map:   0%|          | 0/2450 [00:00<?, ? examples/s]

Map:   0%|          | 0/1050 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 245
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 105
    })
})

In [12]:
# setting pad token
tokenizer.pad_token = tokenizer.eos_token

# data collector
data_collator = transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)

# Fine-tuning Model

In [13]:
# hyperparameters
lr = 2e-4
batch_size = 4
num_epochs = 10

training_args = transformers.TrainingArguments(
    output_dir = "MentalHealth_ft",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    logging_strategy="epoch",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    gradient_accumulation_steps=4,
    warmup_steps=2,
    fp16=True,
    optim="paged_adamw_8bit"
)



In [14]:
trainer = transformers.Trainer(
    model=model,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"],
    args=training_args,
    data_collator=data_collator
)

model.config.use_cache = False
trainer.train()

model.config.use_cache = True

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Epoch,Training Loss,Validation Loss
0,2.325500,2.218588
2,2.002900,2.190331
4,1.456400,2.322835
6,0.978000,2.533627
8,0.668800,2.832974
9,0.539400,2.940861


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.

# Inference

In [18]:
pretrained_model.eval()

# craft prompt
problem = "My son does not listen to me what should i do"
prompt = f"""<s>[INST] <<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
<</SYS>>

{problem} [/INST]"""

inputs = tokenizer(prompt, return_tensors="pt")

outputs = pretrained_model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=500)

print(tokenizer.batch_decode(outputs)[0])

<s><s> [INST] <<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
<</SYS>>

My son does not listen to me what should i do [/INST]

[INST]

[SYS]

[/SYS]

[/INST]
</s>


In [20]:
model.eval()

# craft prompt
prompt = f"""<s>[INST] <<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
<</SYS>>

{problem} [/INST]"""

inputs = tokenizer(prompt, return_tensors="pt")

outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=150)

print(tokenizer.batch_decode(outputs)[0])

<s><s> [INST] <<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
<</SYS>>

My son does not listen to me what should i do [/INST]

It sounds like you are really struggling with your son. It is understandable that you are frustrated and it is important to remember that your son is still a child and he is not able to understand the consequences of his actions.

It may be helpful to seek the support of a therapist. They can help you with strategies to help your son understand the consequences of his actions and help you to set bound

# Save model

In [22]:
# save locally
model.save_pretrained("my-mental-health-model", config=config)

# push to the hub
model.push_to_hub("llama2-mental-health-chat-model", config=config)

# reload
# model = MyModel.from_pretrained("username/my-awesome-model")

adapter_model.safetensors:   0%|          | 0.00/80.0M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Gweizheng/llama2-mental-health-chat-model/commit/f1c234e1a5cbefdf565e71ce9ce7fe20118ab28a', commit_message='Upload model', commit_description='', oid='f1c234e1a5cbefdf565e71ce9ce7fe20118ab28a', pr_url=None, pr_revision=None, pr_num=None)